In [5]:
import pandas as pd
import numpy as np
import config 

In [6]:
# get module variables
ROOT_DIR = config.ROOT_DIR

In [7]:
aw_territories = pd.read_csv(rf"{ROOT_DIR}/storage/processed/aw_territories_dim.csv")

In [8]:
aw_sales_fact = pd.read_csv(rf"{ROOT_DIR}/storage/processed/aw_sales_fact.csv")

In [9]:
aw_returns_fact = pd.read_csv(rf"{ROOT_DIR}/storage/processed/aw_returns_fact.csv")

In [10]:
aw_territories

,SalesTerritoryKey,Region,Country,Continent
0,1,Northwest,United States,North America
1,2,Northeast,United States,North America
2,3,Central,United States,North America
3,4,Southwest,United States,North America
4,5,Southeast,United States,North America
5,6,Canada,Canada,North America
6,7,France,France,Europe
7,8,Germany,Germany,Europe
8,9,Australia,Australia,Pacific
9,10,United Kingdom,United Kingdom,Europe


In [11]:
aw_returns_fact

,ReturnDate,TerritoryKey,ProductKey,ReturnQuantity
0,2015-01-18,9,312,1
1,2015-01-18,10,310,1
2,2015-01-21,8,346,1
3,2015-01-22,4,311,1
4,2015-02-02,6,312,1
...,...,...,...,...
1804,2017-06-30,6,491,1
1805,2017-06-30,8,477,1
1806,2017-06-30,9,477,1
1807,2017-06-30,10,223,1


In [12]:
aw_sales_fact

,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity,QuantityType
0,2015-01-01,2001-09-21,SO45080,332,14657,1,1,1,Single Item
1,2015-01-01,2001-12-05,SO45079,312,29255,4,1,1,Single Item
2,2015-01-01,2001-10-29,SO45082,350,11455,9,1,1,Single Item
3,2015-01-01,2001-11-16,SO45081,338,26782,6,1,1,Single Item
4,2015-01-02,2001-12-15,SO45083,312,14947,10,1,1,Single Item
...,...,...,...,...,...,...,...,...,...
56041,2017-06-30,2004-03-22,SO74143,477,28517,10,3,2,Multiple Items
56042,2017-06-30,2004-03-15,SO74143,479,28517,10,2,1,Single Item
56043,2017-06-30,2004-04-08,SO74143,606,28517,10,1,1,Single Item
56044,2017-06-30,2004-05-15,SO74124,480,21676,7,2,2,Multiple Items


In [13]:
# join returns fact --> territories dim --> sales fact
pd.merge(left=aw_returns_fact, right=aw_territories, how='left', left_on='TerritoryKey', right_on='SalesTerritoryKey').merge(aw_sales_fact, how='left', on="TerritoryKey")

,ReturnDate,TerritoryKey,ProductKey_x,ReturnQuantity,SalesTerritoryKey,Region,Country,Continent,OrderDate,StockDate,OrderNumber,ProductKey_y,CustomerKey,OrderLineItem,OrderQuantity,QuantityType
0,2015-01-18,9,312,1,9,Australia,Australia,Pacific,2015-01-01,2001-10-29,SO45082,350,11455,1,1,Single Item
1,2015-01-18,9,312,1,9,Australia,Australia,Pacific,2015-01-02,2001-12-18,SO45086,314,18747,1,1,Single Item
2,2015-01-18,9,312,1,9,Australia,Australia,Pacific,2015-01-02,2001-10-09,SO45085,312,18746,1,1,Single Item
3,2015-01-18,9,312,1,9,Australia,Australia,Pacific,2015-01-03,2001-10-03,SO45093,312,18906,1,1,Single Item
4,2015-01-18,9,312,1,9,Australia,Australia,Pacific,2015-01-03,2001-10-24,SO45092,313,18899,1,1,Single Item
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15990379,2017-06-30,10,477,1,10,United Kingdom,United Kingdom,Europe,2017-06-30,2004-03-25,SO74108,354,14984,1,1,Single Item
15990380,2017-06-30,10,477,1,10,United Kingdom,United Kingdom,Europe,2017-06-30,2004-06-02,SO74143,462,28517,4,2,Multiple Items
15990381,2017-06-30,10,477,1,10,United Kingdom,United Kingdom,Europe,2017-06-30,2004-03-22,SO74143,477,28517,3,2,Multiple Items
15990382,2017-06-30,10,477,1,10,United Kingdom,United Kingdom,Europe,2017-06-30,2004-03-15,SO74143,479,28517,2,1,Single Item


In [16]:
pd.merge(left=aw_returns_fact, right=aw_territories, how='left', left_on='TerritoryKey', right_on='SalesTerritoryKey').loc[:, ["TerritoryKey", "ReturnQuantity"]].groupby("TerritoryKey").sum()

,ReturnQuantity
TerritoryKey,
1,270
4,362
5,1
6,238
7,186
8,163
9,404
10,204


In [19]:
pd.merge(right=aw_returns_fact, left=aw_territories, how='left', right_on='TerritoryKey', left_on='SalesTerritoryKey').loc[:, ["SalesTerritoryKey", "ReturnQuantity"]].groupby("SalesTerritoryKey").sum()

,ReturnQuantity
SalesTerritoryKey,
1,270.0
2,0.0
3,0.0
4,362.0
5,1.0
6,238.0
7,186.0
8,163.0
9,404.0


In [20]:
# territores + returns fact
a = pd.merge(right=aw_returns_fact, left=aw_territories, how='left', right_on='TerritoryKey', left_on='SalesTerritoryKey').loc[:, ["SalesTerritoryKey", "ReturnQuantity"]].groupby("SalesTerritoryKey").sum()

# territories + sales fact
b = pd.merge(right=aw_sales_fact, left=aw_territories, how='left', right_on='TerritoryKey', left_on='SalesTerritoryKey').loc[:, ["SalesTerritoryKey", "OrderQuantity"]].groupby("SalesTerritoryKey").sum()

In [21]:
b

,OrderQuantity
SalesTerritoryKey,
1,12513
2,40
3,30
4,17191
5,49
6,10894
7,7862
8,7950
9,17951


In [23]:
pd.merge(left=a, right=b, left_index=True, right_index=True)

,ReturnQuantity,OrderQuantity
SalesTerritoryKey,,
1,270.0,12513
2,0.0,40
3,0.0,30
4,362.0,17191
5,1.0,49
6,238.0,10894
7,186.0,7862
8,163.0,7950
9,404.0,17951


#### Transform AdventureWorks_Products.csv from raw to processed

In [126]:
aw_products_dim = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Products.csv")

In [127]:
aw_products_dim.head()

,ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductSize,ProductStyle,ProductCost,ProductPrice
0,214,31,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap...",Red,0,0,13.0863,34.9900
1,215,31,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap...",Black,0,0,12.0278,33.6442
2,218,23,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,Combination of natural and synthetic fibers st...,White,M,U,3.3963,9.5000
3,219,23,SO-B909-L,"Mountain Bike Socks, L",Mountain Bike Socks,Combination of natural and synthetic fibers st...,White,L,U,3.3963,9.5000
4,220,31,HL-U509-B,"Sport-100 Helmet, Blue",Sport-100,"Universal fit, well-vented, lightweight , snap...",Blue,0,0,12.0278,33.6442


In [128]:
aw_products_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ProductKey             293 non-null    int64  
 1   ProductSubcategoryKey  293 non-null    int64  
 2   ProductSKU             293 non-null    object 
 3   ProductName            293 non-null    object 
 4   ModelName              293 non-null    object 
 5   ProductDescription     293 non-null    object 
 6   ProductColor           243 non-null    object 
 7   ProductSize            293 non-null    object 
 8   ProductStyle           293 non-null    object 
 9   ProductCost            293 non-null    float64
 10  ProductPrice           293 non-null    float64
dtypes: float64(2), int64(2), object(7)
memory usage: 25.3+ KB


In [129]:
aw_products_dim.drop(columns="ProductSize", inplace=True)
aw_products_dim.sort_values(by="ProductKey", inplace=True)

In [130]:
# create new column DiscountPrice with 15% discount
aw_products_dim["DiscountPrice"] = aw_products_dim["ProductPrice"] * 0.85

# limit numeric columns ProductCost and ProductPrice to 2 decimal places 
aw_products_dim.loc[:, ["ProductCost", "ProductPrice", "DiscountPrice"]] = aw_products_dim[["ProductCost", "ProductPrice", "DiscountPrice"]].round(2)

In [131]:
# Extract letters before second '-' in ProductSKU column
product_sku_split = aw_products_dim["ProductSKU"].str.split("-", expand=True)
aw_products_dim["SKUType"] = product_sku_split[0].str.cat(product_sku_split[1], sep="-")
aw_products_dim["SKU_category"] = product_sku_split[0]

In [132]:
# replace 0s in ProductStyle column with null values
aw_products_dim["ProductStyle"].replace("0", np.nan, inplace=True)

In [133]:
cond_list = [
    aw_products_dim["ProductPrice"] > 500,
    aw_products_dim["ProductPrice"] > 100,
]
choice_list = [
    "High",
    "Mid-Range"
]

aw_products_dim["price_point"] = np.select(cond_list, choice_list, default="Low")

In [134]:
aw_products_dim.head()

,ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductStyle,ProductCost,ProductPrice,DiscountPrice,SKUType,SKU_category,price_point
0,214,31,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap...",Red,NaN,13.09,34.99,29.74,HL-U509,HL,Low
1,215,31,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap...",Black,NaN,12.03,33.64,28.60,HL-U509,HL,Low
2,218,23,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,Combination of natural and synthetic fibers st...,White,U,3.40,9.50,8.07,SO-B909,SO,Low
3,219,23,SO-B909-L,"Mountain Bike Socks, L",Mountain Bike Socks,Combination of natural and synthetic fibers st...,White,U,3.40,9.50,8.07,SO-B909,SO,Low
4,220,31,HL-U509-B,"Sport-100 Helmet, Blue",Sport-100,"Universal fit, well-vented, lightweight , snap...",Blue,NaN,12.03,33.64,28.60,HL-U509,HL,Low


In [135]:
aw_products_dim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 0 to 292
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ProductKey             293 non-null    int64  
 1   ProductSubcategoryKey  293 non-null    int64  
 2   ProductSKU             293 non-null    object 
 3   ProductName            293 non-null    object 
 4   ModelName              293 non-null    object 
 5   ProductDescription     293 non-null    object 
 6   ProductColor           243 non-null    object 
 7   ProductStyle           209 non-null    object 
 8   ProductCost            293 non-null    float64
 9   ProductPrice           293 non-null    float64
 10  DiscountPrice          293 non-null    float64
 11  SKUType                293 non-null    object 
 12  SKU_category           293 non-null    object 
 13  price_point            293 non-null    object 
dtypes: float64(3), int64(2), object(9)
memory usage: 34.3+ KB


In [136]:
aw_products_dim.to_csv(rf"{ROOT_DIR}/storage/processed/aw_products_dim.csv", index=False)

#### Transform AdventureWorks_Customers.csv from raw to processed

In [137]:
aw_customers_dim = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Customers.csv")

In [138]:
aw_customers_dim.head()

,CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
0,11000,MR.,JON,YANG,4/8/1966,M,M,jon24@adventure-works.com,"$90,000",2,Bachelors,Professional,Y
1,11001,MR.,EUGENE,HUANG,5/14/1965,S,M,eugene10@adventure-works.com,"$60,000",3,Bachelors,Professional,N
2,11002,MR.,RUBEN,TORRES,8/12/1965,M,M,ruben35@adventure-works.com,"$60,000",3,Bachelors,Professional,Y
3,11003,MS.,CHRISTY,ZHU,2/15/1968,S,F,christy12@adventure-works.com,"$70,000",0,Bachelors,Professional,N
4,11004,MRS.,ELIZABETH,JOHNSON,8/8/1968,S,F,elizabeth5@adventure-works.com,"$80,000",5,Bachelors,Professional,Y


In [139]:
aw_customers_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18148 entries, 0 to 18147
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CustomerKey     18148 non-null  int64 
 1   Prefix          18018 non-null  object
 2   FirstName       18148 non-null  object
 3   LastName        18148 non-null  object
 4   BirthDate       18148 non-null  object
 5   MaritalStatus   18148 non-null  object
 6   Gender          18018 non-null  object
 7   EmailAddress    18148 non-null  object
 8   AnnualIncome    18148 non-null  object
 9   TotalChildren   18148 non-null  int64 
 10  EducationLevel  18148 non-null  object
 11  Occupation      18148 non-null  object
 12  HomeOwner       18148 non-null  object
dtypes: int64(2), object(11)
memory usage: 1.8+ MB


In [140]:
# use series string methods with transform to capitalize (proper case) Prefix, FirstName, LastName
aw_customers_dim.loc[:, ["Prefix", "FirstName", "LastName"]] = aw_customers_dim.loc[:, ["Prefix", "FirstName", "LastName"]].transform(lambda x: x.str.capitalize())

In [141]:
# concatenate 
aw_customers_dim["Full_Name"] = aw_customers_dim["Prefix"].str.cat(aw_customers_dim["FirstName"], sep=" ").str.cat(aw_customers_dim["LastName"], sep=" ")

In [142]:
email_split = aw_customers_dim["EmailAddress"].str.split("@", expand=True)
aw_customers_dim["UserName"] = email_split[0]
aw_customers_dim["Domain"] = email_split[1].str.split(".com", expand=True)[0].str.replace("-", " ").str.title()

In [143]:
aw_customers_dim["AnnualIncome"] = aw_customers_dim["AnnualIncome"].str.split("$", expand=True)[1].str.replace(",", "").astype(float)

In [144]:
aw_customers_dim["BirthDate"] = pd.to_datetime(aw_customers_dim["BirthDate"], format="%m/%d/%Y")
aw_customers_dim["Birth_Year"] = aw_customers_dim["BirthDate"].dt.year
aw_customers_dim["Current_Age"] = (datetime.datetime.now() - aw_customers_dim["BirthDate"]).astype("m8[Y]")

In [145]:
aw_customers_dim["Parent"] = np.where(aw_customers_dim["TotalChildren"] > 0, "Yes", "No")

In [146]:
aw_customers_dim["customer_priority"] = np.where((aw_customers_dim["Current_Age"] < 50) & (aw_customers_dim["AnnualIncome"] > 100000), "Priority", "Standard")

In [147]:
aw_customers_dim.head()

,CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner,Full_Name,UserName,Domain,Birth_Year,Current_Age,Parent,customer_priority
0,11000,Mr.,Jon,Yang,1966-04-08,M,M,jon24@adventure-works.com,90000.0,2,Bachelors,Professional,Y,Mr. Jon Yang,jon24,Adventure Works,1966,57.0,Yes,Standard
1,11001,Mr.,Eugene,Huang,1965-05-14,S,M,eugene10@adventure-works.com,60000.0,3,Bachelors,Professional,N,Mr. Eugene Huang,eugene10,Adventure Works,1965,57.0,Yes,Standard
2,11002,Mr.,Ruben,Torres,1965-08-12,M,M,ruben35@adventure-works.com,60000.0,3,Bachelors,Professional,Y,Mr. Ruben Torres,ruben35,Adventure Works,1965,57.0,Yes,Standard
3,11003,Ms.,Christy,Zhu,1968-02-15,S,F,christy12@adventure-works.com,70000.0,0,Bachelors,Professional,N,Ms. Christy Zhu,christy12,Adventure Works,1968,55.0,No,Standard
4,11004,Mrs.,Elizabeth,Johnson,1968-08-08,S,F,elizabeth5@adventure-works.com,80000.0,5,Bachelors,Professional,Y,Mrs. Elizabeth Johnson,elizabeth5,Adventure Works,1968,54.0,Yes,Standard


In [148]:
aw_customers_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18148 entries, 0 to 18147
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CustomerKey        18148 non-null  int64         
 1   Prefix             18018 non-null  object        
 2   FirstName          18148 non-null  object        
 3   LastName           18148 non-null  object        
 4   BirthDate          18148 non-null  datetime64[ns]
 5   MaritalStatus      18148 non-null  object        
 6   Gender             18018 non-null  object        
 7   EmailAddress       18148 non-null  object        
 8   AnnualIncome       18148 non-null  float64       
 9   TotalChildren      18148 non-null  int64         
 10  EducationLevel     18148 non-null  object        
 11  Occupation         18148 non-null  object        
 12  HomeOwner          18148 non-null  object        
 13  Full_Name          18018 non-null  object        
 14  UserNa

In [149]:
aw_customers_dim.to_csv(rf"{ROOT_DIR}/storage/processed/aw_customers_dim.csv", index=False)

#### Transform AdventureWorks_Calendar

In [150]:
aw_calendar_dim = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Calendar.csv")

# convert Date column to datetime[ns] dtype
aw_calendar_dim["Date"] = pd.to_datetime(aw_calendar_dim["Date"], format="%m/%d/%Y")

In [151]:
aw_calendar_dim.head()

,Date
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-04
4,2015-01-05


In [152]:
aw_calendar_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    912 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.2 KB


In [153]:
aw_calendar_dim["Day_Name"] = aw_calendar_dim["Date"].dt.day_name()
aw_calendar_dim["Month_Name"] = aw_calendar_dim["Date"].dt.month_name()
aw_calendar_dim["Year"] = aw_calendar_dim["Date"].dt.year
aw_calendar_dim["Start_of_Year"] = aw_calendar_dim["Date"].dt.to_period("Y").dt.to_timestamp()
aw_calendar_dim["Start_of_Month"] = aw_calendar_dim["Date"].dt.to_period("M").dt.to_timestamp()
aw_calendar_dim["Start_of_Week"] = aw_calendar_dim["Date"].dt.to_period("W").dt.start_time

In [154]:
aw_calendar_dim["Day_Of_Week"] = aw_calendar_dim["Date"].dt.dayofweek + 1 # Monday = 1, Sunday = 7
aw_calendar_dim["Weekend"] = np.where((aw_calendar_dim["Day_Of_Week"] == 6) | (aw_calendar_dim["Day_Of_Week"] == 7), "Weekend", "Weekday")

In [155]:
aw_calendar_dim["short_day"] = aw_calendar_dim["Day_Name"].str.slice(stop=3).str.upper()
aw_calendar_dim["short_month"] = aw_calendar_dim["Month_Name"].str.slice(stop=3).str.upper()

In [156]:
aw_calendar_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            912 non-null    datetime64[ns]
 1   Day_Name        912 non-null    object        
 2   Month_Name      912 non-null    object        
 3   Year            912 non-null    int64         
 4   Start_of_Year   912 non-null    datetime64[ns]
 5   Start_of_Month  912 non-null    datetime64[ns]
 6   Start_of_Week   912 non-null    datetime64[ns]
 7   Day_Of_Week     912 non-null    int64         
 8   Weekend         912 non-null    object        
 9   short_day       912 non-null    object        
 10  short_month     912 non-null    object        
dtypes: datetime64[ns](4), int64(2), object(5)
memory usage: 78.5+ KB


In [157]:
aw_calendar_dim.head()

,Date,Day_Name,Month_Name,Year,Start_of_Year,Start_of_Month,Start_of_Week,Day_Of_Week,Weekend,short_day,short_month
0,2015-01-01,Thursday,January,2015,2015-01-01,2015-01-01,2014-12-29,4,Weekday,THU,JAN
1,2015-01-02,Friday,January,2015,2015-01-01,2015-01-01,2014-12-29,5,Weekday,FRI,JAN
2,2015-01-03,Saturday,January,2015,2015-01-01,2015-01-01,2014-12-29,6,Weekend,SAT,JAN
3,2015-01-04,Sunday,January,2015,2015-01-01,2015-01-01,2014-12-29,7,Weekend,SUN,JAN
4,2015-01-05,Monday,January,2015,2015-01-01,2015-01-01,2015-01-05,1,Weekday,MON,JAN


In [158]:
aw_calendar_dim.to_csv(rf"{ROOT_DIR}/storage/processed/aw_calendar_dim.csv", index=False)

#### Concatenate AdventureWorks_Sales 2015, 2016 and 2017 along vertical axis (equivalent to sql union operation)

In [159]:
aw_sales_2015 = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Sales_2015.csv")
aw_sales_2016 = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Sales_2016.csv")
aw_sales_2017 = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Sales_2017.csv")

In [160]:
aw_sales_fact = pd.concat([aw_sales_2015, aw_sales_2016, aw_sales_2017], ignore_index=True)
aw_sales_fact["QuantityType"] = np.where(aw_sales_fact["OrderQuantity"] > 1, "Multiple Items", "Single Item")

In [161]:
aw_sales_fact["OrderDate"] = pd.to_datetime(aw_sales_fact["OrderDate"], format="%m/%d/%Y")
aw_sales_fact["StockDate"] = pd.to_datetime(aw_sales_fact["StockDate"], format="%m/%d/%Y")

In [162]:
# commenting these transformations and substituting with SUMX dax measure in Power BI to make data model more efficient
# aw_sales_fact = pd.merge(left=aw_sales_fact, right=aw_products_dim.loc[:, ["ProductKey", "ProductPrice"]], how="inner", on="ProductKey")
# aw_sales_fact["Revenue"] = aw_sales_fact["ProductPrice"] * aw_sales_fact["OrderQuantity"]


In [163]:
aw_sales_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56046 entries, 0 to 56045
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OrderDate      56046 non-null  datetime64[ns]
 1   StockDate      56046 non-null  datetime64[ns]
 2   OrderNumber    56046 non-null  object        
 3   ProductKey     56046 non-null  int64         
 4   CustomerKey    56046 non-null  int64         
 5   TerritoryKey   56046 non-null  int64         
 6   OrderLineItem  56046 non-null  int64         
 7   OrderQuantity  56046 non-null  int64         
 8   QuantityType   56046 non-null  object        
dtypes: datetime64[ns](2), int64(5), object(2)
memory usage: 3.8+ MB


In [164]:
aw_sales_fact.to_csv(rf"{ROOT_DIR}/storage/processed/aw_sales_fact.csv", index=False)

#### Transform AdventureWorks_Territories

In [165]:
aw_territories_dim = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Territories.csv")

In [166]:
aw_territories_dim.head()

,SalesTerritoryKey,Region,Country,Continent
0,1,Northwest,United States,North America
1,2,Northeast,United States,North America
2,3,Central,United States,North America
3,4,Southwest,United States,North America
4,5,Southeast,United States,North America


In [167]:
aw_territories_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SalesTerritoryKey  10 non-null     int64 
 1   Region             10 non-null     object
 2   Country            10 non-null     object
 3   Continent          10 non-null     object
dtypes: int64(1), object(3)
memory usage: 448.0+ bytes


In [168]:
aw_territories_dim.to_csv(rf"{ROOT_DIR}/storage/processed/aw_territories_dim.csv", index=False)

#### Transform AdventureWorks_Product_Categories.csv and AdventureWorks_Product_Subcategories.csv

In [169]:
aw_product_categories_dim = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Product_Categories.csv")

In [170]:
aw_product_categories_dim

,ProductCategoryKey,CategoryName
0,1,Bikes
1,2,Components
2,3,Clothing
3,4,Accessories


In [171]:
aw_product_categories_dim.to_csv(rf"{ROOT_DIR}/storage/processed/aw_product_categories_dim.csv", index=False)

In [172]:
aw_product_subcategories_dim = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Product_SubCategories.csv")

In [173]:
aw_product_subcategories_dim.head()

,ProductSubcategoryKey,SubcategoryName,ProductCategoryKey
0,1,Mountain Bikes,1
1,2,Road Bikes,1
2,3,Touring Bikes,1
3,4,Handlebars,2
4,5,Bottom Brackets,2


In [174]:
aw_product_subcategories_dim.to_csv(rf"{ROOT_DIR}/storage/processed/aw_product_subcategories_dim.csv", index=False)

#### Transform AdventureWorks_Returns.csv

In [175]:
aw_returns_fact = pd.read_csv(rf"{ROOT_DIR}/storage/raw/AdventureWorks_Returns.csv")

aw_returns_fact["ReturnDate"] = pd.to_datetime(aw_returns_fact["ReturnDate"], format="%m/%d/%Y")

In [176]:
aw_returns_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1809 entries, 0 to 1808
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ReturnDate      1809 non-null   datetime64[ns]
 1   TerritoryKey    1809 non-null   int64         
 2   ProductKey      1809 non-null   int64         
 3   ReturnQuantity  1809 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 56.7 KB


In [177]:
aw_returns_fact.to_csv(rf"{ROOT_DIR}/storage/processed/aw_returns_fact.csv", index=False)